## Docs
* https://dev.mysql.com/doc/
* https://docs.sqlalchemy.org/en/20/core/engines.html#mysql
* https://jupyter-tutorial.readthedocs.io/en/stable/data-processing/postgresql/ipython-sql.html
* [6.1 Tutorial: Raise Employee's Salary Using a Buffered Cursor](https://dev.mysql.com/doc/connector-python/en/connector-python-tutorial-cursorbuffered.html)

## Ref
* https://towardsdatascience.com/jupyter-magics-with-sql-921370099589

In [1]:
import os
import pandas as pd

import mysql.connector
from dotenv import load_dotenv
from mysql.connector import Error
from sqlalchemy.engine import create_engine

## enviroment

In [2]:
load_dotenv()

_MYSQL_ROOT_USER = os.getenv('MYSQL_ROOT_USER')
_MYSQL_USER = os.getenv('MYSQL_USER')
_MYSQL_ROOT_PASSWORD = os.getenv('MYSQL_ROOT_PASSWORD')
_MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
_MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')
# _MYSQL_USER = os.getenv('MYSQL_USER')
_DB_PORT = os.getenv('DB_PORT')
_DATABASE = os.getenv('DATABASE')

## execution pattern

### Type1: Pandas

In [3]:
URL =f"mysql+pymysql://{_MYSQL_USER}:{_MYSQL_PASSWORD}@{_MYSQL_DATABASE}:{_DB_PORT}/{_DATABASE}"
# URL =f"mysql+pymysql://{_MYSQL_ROOT_USER}:{_MYSQL_ROOT_PASSWORD}@{_MYSQL_DATABASE}:{_DB_PORT}/{_DATABASE}"
engine = create_engine(URL)

In [4]:
df = pd.read_sql('SELECT * FROM Album limit 1000', engine)

In [5]:
len(df)

347

### Type2: magic CMD

In [6]:
# init
%load_ext sql
%config SqlMagic.displaycon=False 
%config SqlMagic.autopandas=True

# IP addresses are variable.　: http://localhost:8888/lab?#%F0%9F%97%92-NOTE
host_ = '172.25.0.2'
%sql mysql+pymysql://{_MYSQL_USER}:{_MYSQL_PASSWORD}@{host_}/{_DATABASE}

#### table_schema

In [7]:
%%sql result_set << 
SELECT table_name, table_schema, table_type
FROM information_schema.tables
WHERE table_schema = 'Chinook'
ORDER BY table_name ASC;

11 rows affected.
Returning data to local variable result_set


In [8]:
result_set

,table_name,table_schema,table_type
0,Album,Chinook,BASE TABLE
1,Artist,Chinook,BASE TABLE
2,Customer,Chinook,BASE TABLE
3,Employee,Chinook,BASE TABLE
4,Genre,Chinook,BASE TABLE
5,Invoice,Chinook,BASE TABLE
6,InvoiceLine,Chinook,BASE TABLE
7,MediaType,Chinook,BASE TABLE
8,Playlist,Chinook,BASE TABLE
9,PlaylistTrack,Chinook,BASE TABLE


#### Table columns
* Album
* Artist

In [9]:
%%sql result_set << 
SHOW COLUMNS FROM Album;

3 rows affected.
Returning data to local variable result_set


In [10]:
result_set

,Field,Type,Null,Key,Default,Extra
0,AlbumId,int(11),NO,PRI,None,
1,Title,varchar(160),NO,,None,
2,ArtistId,int(11),NO,MUL,None,


In [11]:
%%sql result_table << 
SHOW COLUMNS FROM Album;

3 rows affected.
Returning data to local variable result_table


In [12]:
result_table

,Field,Type,Null,Key,Default,Extra
0,AlbumId,int(11),NO,PRI,None,
1,Title,varchar(160),NO,,None,
2,ArtistId,int(11),NO,MUL,None,


In [13]:
%%sql result_table << 
SHOW COLUMNS FROM Artist;

2 rows affected.
Returning data to local variable result_table


In [14]:
result_table

,Field,Type,Null,Key,Default,Extra
0,ArtistId,int(11),NO,PRI,None,
1,Name,varchar(120),YES,,None,


#### INNER JOIN 

In [15]:
%%sql result_set2 <<
SELECT a.AlbumId, b.ArtistId, a.Title, b.Name
FROM
 Album AS a
INNER JOIN Artist AS b ON a.ArtistId = b.ArtistId

347 rows affected.
Returning data to local variable result_set2


In [16]:
# search Artist.Name
result_set2

,AlbumId,ArtistId,Title,Name
0,1,1,For Those About To Rock We Salute You,AC/DC
1,2,2,Balls to the Wall,Accept
2,3,2,Restless and Wild,Accept
3,4,1,Let There Be Rock,AC/DC
4,5,3,Big Ones,Aerosmith
...,...,...,...,...
342,343,226,Respighi:Pines of Rome,Eugene Ormandy
343,344,272,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,345,273,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,274,Mozart: Chamber Music,Nash Ensemble


In [17]:
result_set2.describe()

,AlbumId,ArtistId
count,347.000000,347.000000
mean,174.000000,121.942363
std,100.314505,77.793131
min,1.000000,1.000000
25%,87.500000,58.000000
50%,174.000000,112.000000
75%,260.500000,179.500000
max,347.000000,275.000000


### SELECT
* ArtistId が 指定数より多いレコード
* 自己結合
* JOIN Artist TBL

In [18]:
%%sql result_serect1 <<
-- SET var
SET @artist_id_cnt = 10;

-- Query
SELECT
  c.AlbumId,
  c.ArtistId,
  c.Title,
  d.Name 
FROM 
   (
    SELECT 
      ArtistId,
      COUNT(ArtistId) AS CNT
    FROM Album
    GROUP BY ArtistId
   ) AS ArtistId_CNT
INNER JOIN
  Album AS c
ON
  ArtistId_CNT.ArtistId = c.ArtistId
INNER JOIN
  Artist AS d ON c.ArtistId = d.ArtistId
AND
  ArtistId_CNT.CNT > @artist_id_cnt

0 rows affected.
46 rows affected.
Returning data to local variable result_serect1


In [19]:
result_serect1.head()

,AlbumId,ArtistId,Title,Name
0,30,22,BBC Sessions [Disc 1] [Live],Led Zeppelin
1,43,58,MK III The Final Concerts [Disc 1],Deep Purple
2,44,22,Physical Graffiti [Disc 1],Led Zeppelin
3,50,58,The Final Concerts (Disc 2),Deep Purple
4,58,58,Come Taste The Band,Deep Purple


In [20]:
# Select Artist Name
result_serect1[result_serect1['Name'] == 'Deep Purple']

,AlbumId,ArtistId,Title,Name
1,43,58,MK III The Final Concerts [Disc 1],Deep Purple
3,50,58,The Final Concerts (Disc 2),Deep Purple
4,58,58,Come Taste The Band,Deep Purple
5,59,58,Deep Purple In Rock,Deep Purple
6,60,58,Fireball,Deep Purple
7,61,58,Knocking at Your Back Door: The Best Of Deep P...,Deep Purple
8,62,58,Machine Head,Deep Purple
9,63,58,Purpendicular,Deep Purple
10,64,58,Slaves And Masters,Deep Purple
11,65,58,Stormbringer,Deep Purple


In [21]:
# Max Recode Artist Naem
max_artist_name = max(result_serect1['Name'])
count_ = len(result_serect1['Name'] == max_artist_name)
result = f"Artist: {max_artist_name}, Recode count: {count_}"
result

'Artist: Led Zeppelin, Recode count: 46'

### SET
* album.title

In [22]:
%%sql result_set3 <<
SET @title = '%Rome%';
SELECT * FROM Album WHERE Title LIKE @title

0 rows affected.
3 rows affected.
Returning data to local variable result_set3


In [23]:
result_set3

,AlbumId,Title,ArtistId
0,208,[1997] Black Light Syndrome,136
1,310,Prokofiev: Romeo & Juliet,245
2,343,Respighi:Pines of Rome,226


---

## 🗒 NOTE

### コンテナDB の IP アドレスを検索する
※ コンテナ 再度立ち上げると IP 割り当てかわっている
```bash
# ex:
$ docker inspect {CONTAINER ID} | grep IPAddress
            "SecondaryIPAddresses": null,
            "IPAddress": "",
                    "IPAddress": "172.23.0.2",
```                    